In [58]:
import sympy

We are looking to predict the evolution of mixed invariant vector fields exactly:
    
$\dot{X} = r X + X l$

Which has the solution:
    
$X(t) = e^{rt} X(0) e^{lt}$

However, r and l are not necessary in so(3). It is required that $Ad_X r + l$ and $Ad_X l + r$ are in so(3).

We can easily find the exponential for the right sided vector field for the mixed invariant system. The exponential of this, will not be a group element due to element (3, 4) becoming 1. However, this is cancelled by the $l$ matrix of the mixed-invariant system, and the evolution remains on the manifold.

In [59]:
g = sympy.symbols("g")
r = sympy.Matrix(
    [
        [0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0],
        [0, 0, 0, g, 0],
        [0, 0, 0, 0, -1],
        [0, 0, 0, 0, 0],
    ]
)
sympy.exp(r)

Matrix([
[1, 0, 0, 0,    0],
[0, 1, 0, 0,    0],
[0, 0, 1, g, -g/2],
[0, 0, 0, 1,   -1],
[0, 0, 0, 0,    1]])

The $l$ matrix is not trivial to find a closed form exponential for.

In [60]:
omega_x, omega_y, omega_z, a_x, a_y, a_z = sympy.symbols(
    "omega_x, omega_y, omega_z, a_x, a_y, a_z"
)
l = sympy.Matrix(
    [
        [0, -omega_z, omega_y, 0, 0],
        [omega_z, 0, -omega_x, 0, 0],
        [-omega_y, omega_x, 0, 0, 0],
        [0, 0, 0, 0, -1],
        [0, 0, 0, 0, 0],
    ]
)
l

Matrix([
[       0, -omega_z,  omega_y, 0,  0],
[ omega_z,        0, -omega_x, 0,  0],
[-omega_y,  omega_x,        0, 0,  0],
[       0,        0,        0, 0, -1],
[       0,        0,        0, 0,  0]])

We will analyze this matrix using block matrices. Notice that B is nilpotent. $B^2=0$, $B^3=0$.

In [61]:
A = sympy.MatrixSymbol("A", 3, 2)
B = sympy.MatrixSymbol("B", 2, 2)
Omega = sympy.MatrixSymbol("Omega", 3, 3)
S = sympy.BlockMatrix([[Omega, A], [sympy.ZeroMatrix(2, 3), B]])
S

Matrix([
[Omega, A],
[    0, B]])

We can solve this, since B is nilpotent. To spot the series pattern, we will print several terms.

In [62]:
def find_term(n):
    sub_B_nilpotent = {B**k: sympy.ZeroMatrix(2, 2) for k in range(2, n + 1)}
    return sympy.block_collapse(
        sympy.block_collapse(S**n / sympy.factorial(n)).expand().subs(sub_B_nilpotent)
    )


find_term(0)

I

In [63]:
find_term(1).blocks[0, 1]

A

In [64]:
find_term(2).blocks[0, 1]

(1/2)*A*B + (1/2)*Omega*A

In [65]:
find_term(3).blocks[0, 1]

(1/6)*Omega**2*A + (1/6)*Omega*A*B

In [66]:
find_term(4).blocks[0, 1]

(1/24)*Omega**2*A*B + (1/24)*Omega**3*A

In [67]:
T = sympy.ZeroMatrix(5, 5)
for i in range(5):
    T += find_term(i)
T = sympy.block_collapse(T)
T

Matrix([
[I + (1/2)*Omega**2 + (1/6)*Omega**3 + (1/24)*Omega**4 + Omega, (1/6)*Omega**2*A + (1/24)*Omega**2*A*B + (1/24)*Omega**3*A + A + (1/2)*A*B + (1/2)*Omega*A + (1/6)*Omega*A*B],
[                                                            0,                                                                                                        I + B]])

We now create a series for the top right term.

In [68]:
n = sympy.symbols("n", integer=True)
k = sympy.symbols("k", integer=True)
theta = sympy.symbols("theta")

expr = (Omega ** (k - 1) * A + Omega ** (k - 2) * A * B) / sympy.factorial(k)
expr

1/factorial(k)*(Omega**(k - 2)*A*B + Omega**(k - 1)*A)

The series is only valid after k=2

In [69]:
sympy.block_collapse(expr.subs(k, 2))

(1/2)*A*B + (1/2)*Omega*A

In [70]:
sympy.block_collapse(expr.subs(k, 3))

(1/6)*Omega**2*A + (1/6)*Omega*A*B

In [71]:
sympy.block_collapse(expr.subs(k, 4))

(1/24)*Omega**2*A*B + (1/24)*Omega**3*A

Since Omega is skew symmetric we have that:

In [110]:
sub_omega_powers = {
    Omega ** (2 * k): (-1) ** (k - 1) * theta ** (2 * (k - 1)) * Omega**2,
    Omega ** (2 * k + 1): (-1) ** k * theta ** (2 * k) * Omega,
    Omega ** (2 * k + 2): (-1) ** k * theta ** (2 * k) * Omega**2,
    Omega ** (2 * k + 3): (-1) ** (k + 1) * theta ** (2 * (k + 1)) * Omega,
}
sub_omega_powers

{Omega**(2*k): ((-1)**(k - 1)*theta**(2*k - 2))*Omega**2,
 Omega**(2*k + 1): ((-1)**k*theta**(2*k))*Omega,
 Omega**(2*k + 2): ((-1)**k*theta**(2*k))*Omega**2,
 Omega**(2*k + 3): ((-1)**(k + 1)*theta**(2*k + 2))*Omega}

In [111]:
expr.expand()

1/factorial(k)*Omega**(k - 2)*A*B + 1/factorial(k)*Omega**(k - 1)*A

In [112]:
even_expr = expr.subs(k, 2 * k + 4).subs(sub_omega_powers).expand()
even_expr

((-1)**k*theta**(2*k)/factorial(2*k + 4))*Omega**2*A*B + (-(-1)**k*theta**(2*k + 2)/factorial(2*k + 4))*Omega*A

In [113]:
even_expr.args[0].args[0]

-(-1)**k*theta**(2*k + 2)/factorial(2*k + 4)

In [114]:
even_t1 = sympy.summation(even_expr.args[0].args[0], (k, 0, sympy.oo)).simplify()
even_t1

(-theta**2/2 - cos(theta) + 1)/theta**2

In [115]:
sympy.series(even_t1, theta)

-theta**2/24 + theta**4/720 + O(theta**6)

In [116]:
even_expr.args[1].args[0]

(-1)**k*theta**(2*k)/factorial(2*k + 4)

In [133]:
even_t2 = (
    sympy.summation(even_expr.args[1].args[0], (k, 1, sympy.oo)).simplify().simplify()
)
even_t2

(-theta**4/24 + theta**2/2 + cos(theta) - 1)/theta**4

In [134]:
sympy.series(even_t2, theta)

-theta**2/720 + theta**4/40320 + O(theta**6)

In [216]:
C_1, C_2 = sympy.symbols("C_1, C_2")
C_1_eq = even_t1
C_2_eq = even_t2
even_series_sol = C_1 * Omega * A + C_2 * Omega**2 * A * B
even_series_sol

C_1*Omega*A + C_2*Omega**2*A*B

In [217]:
odd_expr = expr.subs(k, 2 * k + 3).subs(sub_omega_powers).expand()
odd_expr

((-1)**k*theta**(2*k)/factorial(2*k + 3))*Omega**2*A + ((-1)**k*theta**(2*k)/factorial(2*k + 3))*Omega*A*B

In [218]:
odd_expr.args[0].args[0]

(-1)**k*theta**(2*k)/factorial(2*k + 3)

In [219]:
odd_t1 = sympy.summation(odd_expr.args[0].args[0], (k, 0, sympy.oo)).simplify()
odd_t1

(theta - sin(theta))/theta**3

In [220]:
odd_expr.args[1].args[0]

(-1)**k*theta**(2*k)/factorial(2*k + 3)

In [221]:
odd_t2 = sympy.summation(odd_expr.args[1].args[0], (k, 0, sympy.oo)).simplify()
odd_t2

(theta - sin(theta))/theta**3

In [247]:
C_3 = sympy.symbols("C_3")
C_3_eq = odd_t1
odd_series_sol = C_3 * Omega**2 * A + C_3 * Omega * A * B
odd_series_sol

C_3*Omega**2*A + C_3*Omega*A*B

In [248]:
find_term(3).blocks[0, 1]

(1/6)*Omega**2*A + (1/6)*Omega*A*B

In [249]:
N = sympy.block_collapse(
    find_term(1).blocks[0, 1]
    + find_term(2).blocks[0, 1]
    + find_term(3).blocks[0, 1]
    + even_series_sol
    + odd_series_sol
)
sympy.Eq(sympy.MatrixSymbol("N", 3, 2), N)

Eq(N, C_2*Omega**2*A*B + (C_1 + 1/2)*Omega*A + (C_3 + 1/6)*Omega**2*A + (C_3 + 1/6)*Omega*A*B + A + (1/2)*A*B)

This gives us the form of the exponential necessary for mixed-invariant vector fields.

$$ l = \begin{bmatrix} \Omega && A \\ 0 && B \end{bmatrix} $$

$$ exp(l) = \begin{bmatrix} e^{\Omega} && N_l \\ 0 && I + B \end{bmatrix} $$

The r vector also fits this form:
    
$$r = \begin{bmatrix} 0 && C \\ 0 && -B \end{bmatrix}$$

$$exp(r) = \begin{bmatrix}
I && 
N_r  \\
0 && I - B
\end{bmatrix}
$$

$$ X(t) = e^{rt} X(0) e^{lt}$$

$$ N_l \equiv N(\Omega, A, B)$$

$$ N_r \equiv N(0, C, -B)$$

$$ X(t) = \begin{bmatrix}
e^{\Omega_r} && 
N_r  \\
0 && I - B
\end{bmatrix}
\begin{bmatrix}
R && 
P  \\
0 && I
\end{bmatrix}
\begin{bmatrix}
e^{\Omega_l} && 
N_l  \\
0 && I + B
\end{bmatrix} 
$$

$$ X(t) = \begin{bmatrix}
e^{\Omega_r} R && 
e^{\Omega_r}P  + N_r  \\
0 && I - B
\end{bmatrix} \begin{bmatrix}
e^{\Omega_l} && 
N_l  \\
0 && I + B
\end{bmatrix} 
$$

$$ X(t) = \begin{bmatrix}
e^{\Omega_r }R e^{\Omega_l} && 
e^{\Omega_r}R N_l + (e^{\Omega_r} P + N_r)(I + B)\\
0 && I
\end{bmatrix}
$$

Note here that B^2 is nilpotent so that $(I+B)(I-B) = I$.

In [262]:
sub_series = {
    A: sympy.series(A_eq, theta).removeO()
    for A, A_eq in [(C_1, C_1_eq), (C_2, C_2_eq), (C_3, C_3_eq)]
}
sub_series

{C_1: theta**4/720 - theta**2/24,
 C_2: theta**4/40320 - theta**2/720,
 C_3: theta**4/5040 - theta**2/120 + 1/6}

In [263]:
sympy.block_collapse(
    N.subs(sub_series)
    .subs(theta, 0)
    .subs(
        {
            Omega: sympy.ZeroMatrix(3, 3),
            B: sympy.Matrix([[0, 0], [0, 1]]),
            A: sympy.Matrix([[0, 0], [0, 0], [g, 0]]),
        }
    )
)

Matrix([
[0, 0],
[0, 0],
[g, 0]])

In [264]:
even_t2.series()

-theta**2/720 + theta**4/40320 + O(theta**6)

In [265]:
N.series(theta)

C_2*Omega**2*A*B + (C_1 + 1/2)*Omega*A + (C_3 + 1/6)*Omega**2*A + (C_3 + 1/6)*Omega*A*B + A + (1/2)*A*B